In [1]:
'''Исходная программа игры претерпела следующие изменения:

    1. Введена стартовая функция с выбором режима игры. 
        Режимы: выход из игры - "exit", 
                просто игра - "start",
                разовый тест алгоритма - "single test",
                многоразовый тест - "1000 test"
    2. Добавлена возможность задать пользовательский диапазон, в котором будет находиться загаданное число.
    3. Добавлена возможность завершить игру в любой момент простой игры.
    
    4. Логика поиска числа при тестировании:
        Шаг 1. Получаем границы интервала.
        Шаг 2. Находим середину интервала (СИ) и возвращаем СИ в качестве угадываемого числа.
            (Начало интервала + Конец интервала) / 2
        Шаг 3. Получаем ответ от игры. Если не угадали, то переопределяем границы интервала 
            от СИ до старой границы так, чтобы угадыаваемое число попадало в интервал. 
            Возвращаем новые границы и переходим к Шагу 2.'''

'''Для запуска игры используйте функцию "game_start()"'''

borders_and_digit = [] # Списокхранит загаданное число и границы диапазона.
show_message = True    # Нужна для включения/выключения вывода сообщений.


def game_start():
    '''Функция принимает ответ пользователя и запускает/останавливает/тестирует игру.'''
    user_answer = ''
    global show_message
    #show_message = True  # Сообщения будут выводиться
    
    # Цикл работает до тех пор, пока пользователь не введёт любую команду
    while user_answer != 'exit':
        
        print("Введите команду 'start' для начала игры.\n",
              "Введите команду 'exit' для завершения игры.\n",
              "Введите команду 'single test'/'1000 test' для тестирования алгоритма.", sep='\n'
             )
        user_answer = input()    # Пользователь выбирает режим
        user_answer = user_answer.lower()
        
        if user_answer == 'start':
            show_message = True  # Сообщения будут выводиться
            user_guess_number(game_pick_digit(user_set_range())) # Начинает игру "Угадай число"
            return
        elif user_answer == 'single test':
            show_message = True  # Сообщения будут выводиться
            game_pick_digit(user_set_range())
            core_auto_guess(borders_and_digit) # Начинает разовый тест алгоритма
            return   
        elif user_answer == '1000 test':
            show_message = False # Сообщения НЕ будут выводиться
            score_game()         # Начинает многоразовый тест алгоритма
            return   
    
    return
                        

        
def user_set_range():
    '''Функция для ввода пользователем двух целых чисел, 
    определяющих границы интервала для загадываемого числа.'''
    import re
    
    pattern = re.compile('-?\d+\,\s\-?\d+')   # Паттерн для проверки введённого пользователем ответа.
    user_answer = ' '    
    range_border = []
    
    # Цикл повторяется до тех пор, пока ответ не совпадёт с паттерном или пользователь не завершит игру
    while pattern.match(user_answer) == None:
        if user_answer.lower() == 'exit': return('exit') # Завершение игры.
        else: 
            print("Оставьте поле пустым, если хотите интервал по умолчанию.\n", 
                  "Введите два любых целых числа через запятую: "
                 )
            user_answer = input()
            if user_answer == '': return([1, 100])
    
    for number in user_answer.split(', '):
        range_border.append(int(number))      # Переводим ответ пользователя в числа
    range_border.sort()
    
    return range_border



def game_pick_digit(range_border):
    '''Загадывает число в заданном пользователем интервале.'''
    if range_border == 'exit': return('exit') # Досрочное завершение игры
    global show_message
    
    import numpy
        
    border_open = int(range_border[0])
    border_close = int(range_border[-1])
    digit = numpy.random.randint(border_open, border_close+1) # Загадали число.
    
    if show_message == True: print (f"Загадано число от {border_open} до {border_close}")
    
    # Обновление глобального списка с границами диапазона и загаданным числом
    global borders_and_digit
    borders_and_digit = []
    borders_and_digit.append(border_open)
    borders_and_digit.append(digit)
    borders_and_digit.append(border_close+1)
    # Раскомментируйте нижние строки, чтобы увидеть списки
    #print(borders_and_digit)
    return digit



def user_guess_number(digit):
    '''Пользователь угадывает число.'''
    if digit == 'exit': return('exit')   # Досрочное завершение игры
    
    counter = 1                          # Счетчик попыток.
    guess = ''                           # Предполагаемое число.

    # Цикл повторяется до тех пор, пока ответ не совпадёт с загаданным числом или пользователь не завершит игру
    while guess.lower() != 'exit':
        guess = input('Введите число: ') # Предполагаемое число.
        
        try:
            if digit == int(guess):
                print (f"Вы угадали число {digit} за {counter} попыток.")
                break    # Выход из цикла, если угадали.
            elif digit > int(guess): print (f"Угадываемое число больше {guess}.")
            elif digit < int(guess): print (f"Угадываемое число меньше {guess}.")
        except: continue # Опечатка не считается за попытку
            
        counter += 1

    return



def core_auto_guess(borders_and_digit):
    '''Функция автоматически подбирает загаданное число из диапазона, 
    выводит уведомление об одгаданном числе и количестве попыток. Возвращает число попыток.
    С каждой итерацией диапазон сужается вдвое и положение числа уточняется.'''
    if borders_and_digit == 'exit': return('exit')     # Досрочное завершение игры
    global show_message
    
    digit = int(borders_and_digit[1])                  # Загаданное число
    range_open = int(borders_and_digit[0])             # Открывающая граница интервала
    range_close = int(borders_and_digit[-1])           # Закрывающая граница интервала
    range_center = int((range_open + range_close) / 2) # Предполагаемое число
    
    counter = 1                                        # Счетчик попыток.
    
    # Цикл повторяется до тех пор, пока ответ не совпадёт с загаданным числом или пользователь не завершит игру
    while digit != range_center:
        counter += 1 
        if digit > range_center:
            range_open = range_center  # Заменяем открывающую границу на предполагаемое число
            range_center = int((range_open + range_close) / 2) # Обновляем предполагаемое число
        elif digit < range_center: 
            range_close = range_center # Заменяем закрывающую границу на предполагаемое число
            range_center = int((range_open + range_close) / 2) # Обновляем предполагаемое число
        
    if show_message == True: print (f"Компьютер угадал {digit} за {counter} попыток.")
    
    return counter



def score_game(size=1000):
    '''Запускаем игру 1000 раз, чтобы узнать, как быстро игра угадывает число.'''
    import numpy
    game_pick_digit(user_set_range()) # Пользователь вводит границы интервала
    #global show_message
    #show_message = False # Сообщения НЕ будут выводиться
    
    # Сохраняем границы интервала в перманентном списке
    borders = []
    borders.append(borders_and_digit[0])
    borders.append(borders_and_digit[-1])
    
    count_ls = []
    
    numpy.random.seed(1) # фиксируем RANDOM SEED, чтобы ваш эксперимент был воспроизводим!
    random_array = numpy.random.randint(int(borders[0]), int(borders[-1])+1, size=(1000))
    for number in random_array:
        count_ls.append(core_auto_guess(borders_and_digit))
        game_pick_digit(borders)
    score = int(numpy.mean(count_ls))
    print(f"Ваш алгоритм угадывает число в среднем за {score} попыток.")
    # Раскомментируйте нижние строки, чтобы увидеть списки
    #print(count_ls)
    #print(borders_and_digit)
    return(score)

In [5]:
"""Запустите игру нажав Enter"""
game_start()

Введите команду 'start' для начала игры.

Введите команду 'exit' для завершения игры.

Введите команду 'single test'/'1000 test' для тестирования алгоритма.
1000 test
Оставьте поле пустым, если хотите интервал по умолчанию.
 Введите два любых целых числа через запятую: 

Ваш алгоритм угадывает число в среднем за 5 попыток.
